# 🌎 Welcome to the CSE151B Spring 2025 Climate Emulation Competition!

Thank you for participating in this exciting challenge focused on building machine learning models to emulate complex climate systems.  
This notebook is provided as a **starter template** to help you:

- Understand how to load and preprocess the dataset  
- Construct a baseline model  
- Train and evaluate predictions using a PyTorch Lightning pipeline  
- Format your predictions for submission to the leaderboard  

You're encouraged to:
- Build on this structure or replace it entirely
- Try more advanced models and training strategies
- Incorporate your own ideas to push the boundaries of what's possible

If you're interested in developing within a repository structure and/or use helpful tools like configuration management (based on Hydra) and logging (with Weights & Biases), we recommend checking out the following Github repo. Such a structure can be useful when running multiple experiments and trying various research ideas.

👉 [https://github.com/salvaRC/cse151b-spring2025-competition](https://github.com/salvaRC/cse151b-spring2025-competition)

Good luck, have fun, and we hope you learn a lot through this process!


### 📦 Install Required Libraries
We install the necessary Python packages for data loading, deep learning, and visualization.


In [ ]:
!pip install xarray zarr dask lightning matplotlib wandb cftime einops nc-time-axis --quiet

import os
from datetime import datetime
import numpy as np
import xarray as xr
import nc_time_axis
import dask.array as da
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import lightning.pytorch as pl
import torch.nn.functional as F
import math
import pandas as pd
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from lightning.pytorch import Trainer

In [ ]:
! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c cse151b-spring2025-competition
! mkdir data
! unzip cse151b-spring2025-competition.zip -d data

### ⚙️ Configuration Setup  
Define all model, data, and training hyperparameters in one place for easy control and reproducibility.

### 📊 Data Configuration

We define the dataset settings used for training and evaluation. This includes:

- **`path`**: Path to the `.zarr` dataset containing monthly climate variables from CMIP6 simulations.
- **`input_vars`**: Climate forcing variables (e.g., CO₂, CH₄) used as model inputs.
- **`output_vars`**: Target variables to predict — surface air temperature (`tas`) and precipitation (`pr`).
- **`target_member_id`**: Ensemble member to use from the simulations (each SSP has 3) for target variables.
- **`train_ssps`**: SSP scenarios used for training (low to high emissions).
- **`test_ssp`**: Scenario held out for evaluation (Must be set to SSP245).
- **`test_months`**: Number of months to include in the test split (Must be set to 120).
- **`batch_size`** and **`num_workers`**: Data loading parameters for PyTorch training.

These settings reflect how the challenge is structured: models must learn from some emission scenarios and generalize to unseen ones.

> ⚠️ **Important:** Do **not modify** the following test settings:
>
> - `test_ssp` must remain **`ssp245`**, which is the held-out evaluation scenario.
> - `test_months` must be **`120`**, corresponding to the last 10 years (monthly resolution) of the scenario.



In [ ]:
#NOTE Change the data directory according to where you have your zarr files stored
config = {
    "data": {
        "path": "data/processed_data_cse151b_v2_corrupted_ssp245/processed_data_cse151b_v2_corrupted_ssp245.zarr",
        "input_vars": ["CO2", "SO2", "CH4", "BC", "rsdt"],
        "output_vars": ["tas", "pr"],
        "target_member_id": 0,
        "train_ssps": ["ssp126", "ssp370", "ssp585"],
        "test_ssp": "ssp245",
        "test_months": 360,
        "batch_size": 16,
        "num_workers": 4,
    },

    "model": {
    "type": "conv_lstm",  # <- changed from "cnn_transformer"
    "cnn_init_dim": 64,
    "cnn_depth": 4,
    "dropout_rate": 0.2,
  },

    "training": {
        "lr": 1e-3,
    },
    "trainer": {
        "max_epochs": 50,
        "accelerator": "auto",
        "devices": "auto",
        "precision": 32,
        "deterministic": True,
        "num_sanity_val_steps": 0,
    },
    "seed": 42,
}
pl.seed_everything(config["seed"])
torch.use_deterministic_algorithms(True, warn_only=True)  # Set seed for reproducibility

## EDA

In [ ]:
# Load the data (update the path to the correct location)
data_path = "data/processed_data_cse151b_v2_corrupted_ssp245/processed_data_cse151b_v2_corrupted_ssp245.zarr"
data = xr.open_zarr(data_path)
data

In [ ]:
def get_lat_weights(latitude_values):
    lat_rad = np.deg2rad(latitude_values)
    weights = np.cos(lat_rad)
    return weights / np.mean(weights)

### 🧠 SimpleCNN: A Residual Convolutional Baseline

This is a lightweight baseline model designed to capture spatial patterns in global climate data using convolutional layers.

- The architecture starts with a **convolution + batch norm + ReLU** block to process the input channels.
- It then applies a series of **residual blocks** to extract increasingly abstract spatial features. These help preserve gradient flow during training.
- Finally, a few convolutional layers reduce the feature maps down to the desired number of output channels (`tas` and `pr`).

This model only serves as a **simple baseline for climate emulation**.

We encourage you to build and experiment with your own models and ideas.


In [ ]:
class DoubleConv(nn.Module):
    """Double convolution block used in U-Net"""
    def __init__(self, in_channels, out_channels, dropout_rate=0.0):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Dropout2d(dropout_rate),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""
    def __init__(self, in_channels, out_channels, dropout_rate=0.0):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels, dropout_rate)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""
    def __init__(self, in_channels, out_channels, bilinear=True, dropout_rate=0.0):
        super().__init__()

        # Use bilinear upsampling or transposed conv
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            # The input will be concatenated with skip connection
            # So we need to handle: in_channels (from deeper) + skip_channels -> out_channels
            self.conv = DoubleConv(in_channels, out_channels, dropout_rate)
        else:
            self.up = nn.ConvTranspose2d(in_channels // 2, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels, dropout_rate)

    def forward(self, x1, x2):
        # x1 is from deeper layer (to be upsampled)
        # x2 is skip connection from encoder
        x1 = self.up(x1)

        # Handle size mismatch
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])

        # Concatenate skip connection with upsampled features
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class SpatialAttention(nn.Module):
    """Spatial attention mechanism for U-Net"""
    def __init__(self, channels):
        super().__init__()
        self.query_conv = nn.Conv2d(channels, channels // 8, kernel_size=1)
        self.key_conv = nn.Conv2d(channels, channels // 8, kernel_size=1)
        self.value_conv = nn.Conv2d(channels, channels, kernel_size=1)
        self.scale = math.sqrt(channels // 8 + 1e-5)

    def forward(self, x):
        batch_size, channels, height, width = x.size()
        query = self.query_conv(x).view(batch_size, -1, height * width).permute(0, 2, 1)
        key = self.key_conv(x).view(batch_size, -1, height * width)
        value = self.value_conv(x).view(batch_size, channels, -1)

        attention = torch.bmm(query, key) / self.scale
        attention = F.softmax(attention, dim=-1)

        context = torch.bmm(value, attention.permute(0, 2, 1))
        context = context.view(batch_size, channels, height, width)
        return context + x

class ConvLSTMCell(nn.Module):
  def __init__(self, input_dim, hidden_dim, kernel_size=3):
      super().__init__()
      self.hidden_dim = hidden_dim
      padding = kernel_size // 2
      self.conv = nn.Conv2d(
          input_dim + hidden_dim, 4 * hidden_dim,
          kernel_size=kernel_size, padding=padding
      )

  def forward(self, x, state):
      h_cur, c_cur = state
      combined = torch.cat([x, h_cur], dim=1)  # Concatenate along channel dimension
      conv_out = self.conv(combined)
      cc_i, cc_f, cc_o, cc_g = torch.chunk(conv_out, 4, dim=1)

      i = torch.sigmoid(cc_i)
      f = torch.sigmoid(cc_f)
      o = torch.sigmoid(cc_o)
      g = torch.tanh(cc_g)

      c_next = f * c_cur + i * g
      h_next = o * torch.tanh(c_next)

      return h_next, c_next



class ClimateUNet(pl.LightningModule):
    def __init__(
        self,
        n_input_channels,
        n_output_channels,
        base_channels=64,
        depth=4,
        bilinear=True,
        use_attention=True,
        attention_levels=[1, 2],  # Which levels to apply attention
        dropout_rate=0.2,
        learning_rate=1e-4,
        weight_decay=1e-5,
        l1_reg=1e-5
    ):
        super().__init__()
        self.save_hyperparameters()
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.l1_reg = l1_reg
        self.depth = depth
        self.use_attention = use_attention
        self.attention_levels = attention_levels

        # Calculate channel sizes for each level
        self.encoder_channels = [base_channels * (2 ** i) for i in range(depth + 1)]

        # Initial convolution
        self.inc = DoubleConv(n_input_channels, base_channels, dropout_rate)

        # Encoder (downsampling path)
        self.down_layers = nn.ModuleList()
        self.attention_layers = nn.ModuleList()

        for i in range(depth):
            in_ch = self.encoder_channels[i]
            out_ch = self.encoder_channels[i + 1]
            self.down_layers.append(Down(in_ch, out_ch, dropout_rate))

            # Add attention at specified levels
            if use_attention and i in attention_levels:
                self.attention_layers.append(SpatialAttention(out_ch))
            else:
                self.attention_layers.append(nn.Identity())

        # Decoder (upsampling path)
        self.up_layers = nn.ModuleList()
        for i in range(depth):
            # Going backwards through encoder channels
            deep_ch = self.encoder_channels[depth - i]      # From deeper layer
            skip_ch = self.encoder_channels[depth - i - 1]  # From skip connection
            out_ch = skip_ch  # Output should match skip connection size

            # Input to Up layer is deep_ch + skip_ch after concatenation
            concat_ch = deep_ch + skip_ch
            self.up_layers.append(Up(concat_ch, out_ch, bilinear, dropout_rate))

        # Output convolution
        self.outc = nn.Conv2d(base_channels, n_output_channels, kernel_size=1)

    def forward(self, x):
        # Initial convolution
        x = self.inc(x)

        # Store skip connections
        skip_connections = [x]

        # Encoder path
        for i, (down, attention) in enumerate(zip(self.down_layers, self.attention_layers)):
            x = down(x)
            x = attention(x)  # Apply attention if configured
            skip_connections.append(x)

        # Remove the last element (bottleneck) as we'll use it as input to decoder
        x = skip_connections.pop()

        # Decoder path
        for up in self.up_layers:
            skip = skip_connections.pop()
            x = up(x, skip)

        # Output
        logits = self.outc(x)
        return logits

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
            self.parameters(),
            lr=self.learning_rate,
            weight_decay=self.weight_decay
        )
        scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer, T_0=10, T_mult=1, eta_min=1e-6
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {"scheduler": scheduler, "interval": "epoch", "frequency": 1},
            "monitor": "val_loss"
        }

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)

        # L1 regularization for stability
        stability_loss = sum(
            self.l1_reg * torch.sum(torch.abs(p))
            for n, p in self.named_parameters()
            if 'weight' in n and 'bn' not in n
        )
        total_loss = loss + stability_loss

        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log('stability_loss', stability_loss, on_epoch=True)
        return total_loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        self.log('val_loss', loss, on_epoch=True, prog_bar=True)

        # Log individual variable losses if multi-output
        if y.shape[1] >= 2:
            self.log('val_tas_loss', F.mse_loss(y_hat[:, 0:1], y[:, 0:1]), on_epoch=True)
            self.log('val_pr_loss', F.mse_loss(y_hat[:, 1:2], y[:, 1:2]), on_epoch=True)

        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        self.log('test_loss', loss)
        return loss

    def on_before_optimizer_step(self, optimizer):
        # Gradient clipping for stability
        torch.nn.utils.clip_grad_norm_(self.parameters(), max_norm=1.0)


class TemporalUNet(ClimateUNet):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        # Match channels to encoder output levels
        self.temporal_blocks = nn.ModuleList([
            ConvLSTMCell(self.encoder_channels[-1], self.encoder_channels[-1]),
        ])

    def forward(self, x_seq):
        """
        x_seq shape: [B, T, C, H, W]
        Applies encoder over each timestep, ConvLSTM on encoded outputs, then decoder
        """
        B, T, C, H, W = x_seq.shape

        # First encode each timestep
        encoded_sequence = []
        for t in range(T):
            x_t = x_seq[:, t]
            x_t = self.inc(x_t)
            skip_connections = [x_t]
            for i, (down, attention) in enumerate(zip(self.down_layers, self.attention_layers)):
                x_t = down(x_t)
                x_t = attention(x_t)
                skip_connections.append(x_t)
            encoded_sequence.append(skip_connections)

        # Stack final encoder outputs: [T, B, C, H, W]
        bottleneck_sequence = torch.stack([enc[-1] for enc in encoded_sequence], dim=0)

        # Initialize ConvLSTM state
        h = torch.zeros_like(bottleneck_sequence[0])
        c = torch.zeros_like(bottleneck_sequence[0])

        for t in range(T):
            h, c = self.temporal_blocks[0](bottleneck_sequence[t], (h, c))

        # Now decode last timestep using skip connections from last input
        x = h
        skip_connections = encoded_sequence[-1][:-1][::-1]  # skip connections (reversed)
        for up, skip in zip(self.up_layers, skip_connections):
            x = up(x, skip)

        return self.outc(x)



def create_climate_unet(
    n_input_channels,
    n_output_channels,
    base_channels=64,
    depth=4,
    bilinear=True,
    use_attention=True,
    attention_levels=None,
    dropout_rate=0.2,
    learning_rate=1e-4,
    weight_decay=1e-5,
    l1_reg=1e-5
):
    """
    Factory function to create a Climate U-Net model

    Args:
        n_input_channels: Number of input channels (climate variables)
        n_output_channels: Number of output channels (tas, pr)
        base_channels: Base number of channels (will be doubled at each level)
        depth: Number of down/up sampling levels
        bilinear: Use bilinear upsampling instead of transposed convolution
        use_attention: Whether to use spatial attention
        attention_levels: List of levels where to apply attention (0-indexed)
        dropout_rate: Dropout rate for regularization
        learning_rate: Learning rate for optimizer
        weight_decay: Weight decay for regularization
        l1_reg: L1 regularization coefficient
    """
    if attention_levels is None:
        attention_levels = [1, 2]  # Apply attention at middle levels

    return TemporalUNet(
        n_input_channels=n_input_channels,
        n_output_channels=n_output_channels,
        base_channels=base_channels,
        depth=depth,
        bilinear=bilinear,
        use_attention=use_attention,
        attention_levels=attention_levels,
        dropout_rate=dropout_rate,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        l1_reg=l1_reg
    )


### 📐 Normalizer: Z-Score Scaling for Climate Inputs & Outputs

This class handles **Z-score normalization**, a crucial preprocessing step for stable and efficient neural network training:

- **`set_input_statistics(mean, std)` / `set_output_statistics(...)`**: Store the mean and standard deviation computed from the training data for later use.
- **`normalize(data, data_type)`**: Standardizes the data using `(x - mean) / std`. This is applied separately to inputs and outputs.
- **`inverse_transform_output(data)`**: Converts model predictions back to the original physical units (e.g., Kelvin for temperature, mm/day for precipitation).

Normalizing the data ensures the model sees inputs with similar dynamic ranges and avoids biases caused by different variable scales.


In [ ]:
class Normalizer:
    def __init__(self):
        self.mean_in, self.std_in = None, None
        self.mean_out, self.std_out = None, None

    def set_input_statistics(self, mean, std):
        self.mean_in = mean
        self.std_in = std

    def set_output_statistics(self, mean, std):
        self.mean_out = mean
        self.std_out = std

    def normalize(self, data, data_type):
        if data_type == "input":
            return (data - self.mean_in) / self.std_in
        elif data_type == "output":
            return (data - self.mean_out) / self.std_out

    def inverse_transform_output(self, data):
        return data * self.std_out + self.mean_out


### 🌍 Data Module: Loading, Normalization, and Splitting

This section handles the entire data pipeline, from loading the `.zarr` dataset to preparing PyTorch-ready DataLoaders.

#### `ClimateDataset`
- A simple PyTorch `Dataset` wrapper that preloads the entire (normalized) dataset into memory using Dask.
- Converts the data to PyTorch tensors and handles any `NaN` checks up front.

#### `ClimateDataModule`
A PyTorch Lightning `DataModule` that handles:
- ✅ **Loading data** from different SSP scenarios and ensemble members
- ✅ **Broadcasting non-spatial inputs** (like CO₂) to match spatial grid size
- ✅ **Normalization** using mean/std computed from training data only
- ✅ **Splitting** into training, validation, and test sets:
  - Training: All months from selected SSPs (except last 10 years of SSP370)
  - Validation: Last 10 years (120 months) of SSP370
  - Test: Last 10 years of SSP245 (unseen scenario)
- ✅ **Batching** and parallelized data loading via PyTorch `DataLoader`s
- ✅ **Latitude-based area weighting** for fair climate metric evaluation
- Shape of the inputs are Batch_Size X 5 (num_input_variables) X 48 X 72
- Shape of ouputputs are Batch_Size X 2 (num_output_variables) X 48 X 72

> ℹ️ **Note:** You likely won’t need to modify this class but feel free to make modifications if you want to inlcude different ensemble mebers to feed more data to your models


In [ ]:
class ClimateDataset(Dataset):
    def __init__(self, inputs_dask, outputs_dask, output_is_normalized=True):
        self.size = inputs_dask.shape[0]
        print(f"Creating dataset with {self.size} samples...")

        inputs_np = inputs_dask.compute()
        outputs_np = outputs_dask.compute()

        self.inputs = torch.from_numpy(inputs_np).float()
        self.outputs = torch.from_numpy(outputs_np).float()

        if torch.isnan(self.inputs).any() or torch.isnan(self.outputs).any():
            raise ValueError("NaNs found in dataset")

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        return self.inputs[idx], self.outputs[idx]


class ClimateDataModule(pl.LightningDataModule):
    def __init__(
        self,
        path,
        input_vars,
        output_vars,
        train_ssps,
        test_ssp,
        target_member_id,
        val_split=0.1,
        test_months=120,
        batch_size=32,
        num_workers=0,
        seed=42,
    ):
        super().__init__()
        self.path = path
        self.input_vars = input_vars
        self.output_vars = output_vars
        self.train_ssps = train_ssps
        self.test_ssp = test_ssp
        self.target_member_id = target_member_id
        self.val_split = val_split
        self.test_months = test_months
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.seed = seed
        self.normalizer = Normalizer()

    def prepare_data(self):
        assert os.path.exists(self.path), f"Data path not found: {self.path}"

    def setup(self, stage=None):
        ds = xr.open_zarr(self.path, consolidated=False, chunks={"time": 24})
        spatial_template = ds["rsdt"].isel(time=0, ssp=0, drop=True)

        def load_ssp(ssp):
            input_dask, output_dask = [], []
            for var in self.input_vars:
                da_var = ds[var].sel(ssp=ssp)
                if "latitude" in da_var.dims:
                    da_var = da_var.rename({"latitude": "y", "longitude": "x"})
                if "member_id" in da_var.dims:
                    da_var = da_var.sel(member_id=self.target_member_id)
                if set(da_var.dims) == {"time"}:
                    da_var = da_var.broadcast_like(spatial_template).transpose("time", "y", "x")
                input_dask.append(da_var.data)

            for var in self.output_vars:
                da_out = ds[var].sel(ssp=ssp, member_id=self.target_member_id)
                if "latitude" in da_out.dims:
                    da_out = da_out.rename({"latitude": "y", "longitude": "x"})
                output_dask.append(da_out.data)

            return da.stack(input_dask, axis=1), da.stack(output_dask, axis=1)

        train_input, train_output, val_input, val_output = [], [], None, None

        for ssp in self.train_ssps:
            x, y = load_ssp(ssp)
            if ssp == "ssp370":
                val_input = x[-self.test_months:]
                val_output = y[-self.test_months:]
                train_input.append(x[:-self.test_months])
                train_output.append(y[:-self.test_months])
            else:
                train_input.append(x)
                train_output.append(y)

        train_input = da.concatenate(train_input, axis=0)
        train_output = da.concatenate(train_output, axis=0)

        self.normalizer.set_input_statistics(
            mean=da.nanmean(train_input, axis=(0, 2, 3), keepdims=True).compute(),
            std=da.nanstd(train_input, axis=(0, 2, 3), keepdims=True).compute(),
        )
        self.normalizer.set_output_statistics(
            mean=da.nanmean(train_output, axis=(0, 2, 3), keepdims=True).compute(),
            std=da.nanstd(train_output, axis=(0, 2, 3), keepdims=True).compute(),
        )

        train_input_norm = self.normalizer.normalize(train_input, "input")
        train_output_norm = self.normalizer.normalize(train_output, "output")
        val_input_norm = self.normalizer.normalize(val_input, "input")
        val_output_norm = self.normalizer.normalize(val_output, "output")

        test_input, test_output = load_ssp(self.test_ssp)
        test_input = test_input[-self.test_months:]
        test_output = test_output[-self.test_months:]
        test_input_norm = self.normalizer.normalize(test_input, "input")

        self.train_dataset = ClimateDataset(train_input_norm, train_output_norm)
        self.val_dataset = ClimateDataset(val_input_norm, val_output_norm)
        self.test_dataset = ClimateDataset(test_input_norm, test_output, output_is_normalized=False)

        self.lat = spatial_template.y.values
        self.lon = spatial_template.x.values
        self.area_weights = xr.DataArray(get_lat_weights(self.lat), dims=["y"], coords={"y": self.lat})

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True,
                          num_workers=self.num_workers, pin_memory=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False,
                          num_workers=self.num_workers, pin_memory=True)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False,
                          num_workers=self.num_workers, pin_memory=True)

    def get_lat_weights(self):
        return self.area_weights

    def get_coords(self):
        return self.lat, self.lon

### ⚡ ClimateEmulationModule: Lightning Wrapper for Climate Model Emulation

This is the core model wrapper built with **PyTorch Lightning**, which organizes the training, validation, and testing logic for the climate emulation task. Lightning abstracts away much of the boilerplate code in PyTorch-based deep learning workflows, making it easier to scale models.

#### ✅ Key Features

- **`training_step` / `validation_step` / `test_step`**: Standard Lightning hooks for computing loss and predictions at each stage. The loss used is **Mean Squared Error (MSE)**.

- **Normalization-aware outputs**:
  - During validation and testing, predictions and targets are denormalized before evaluation using stored mean/std statistics.
  - This ensures evaluation is done in real-world units (Kelvin and mm/day).

- **Metric Evaluation** via `_evaluate()`:
  For each variable (`tas`, `pr`), it calculates:
  - **Monthly Area-Weighted RMSE**
  - **Time-Mean RMSE** (RMSE on 10-year average's)
  - **Time-Stddev MAE** (MAE on 10-year standard deviation; a measure of temporal variability)
    
  These metrics reflect the competition's evaluation criteria and are logged and printed.

- **Kaggle Submission Writer**:
  After testing, predictions are saved to a `.csv` file in the required Kaggle format via `_save_submission()`.

- **Saving Predictions for Visualization**:
  - Validation predictions are saved tao `val_preds.npy` and `val_trues.npy`
  - These can be loaded later for visual inspection of the model's performance.

 🔧 **Feel free to modify any part of this module** (loss functions, evaluation, training logic) to better suit your model or training pipeline / Use pure PyTorch etc.

⚠️ The **final submission `.csv` file must strictly follow the format and naming convention used in `_save_submission()`**, as these `ID`s are used to match predictions to the hidden test set during evaluation.



In [ ]:
class PhysicsAwareLoss(nn.Module):
    def __init__(self, mse_weight=1.0, gradient_weight=0.1, spectral_weight=0.1):
        super().__init__()
        self.mse_weight = mse_weight
        self.gradient_weight = gradient_weight
        self.spectral_weight = spectral_weight

    def forward(self, pred, target):
        # Standard MSE
        mse_loss = F.mse_loss(pred, target)

        # Spatial gradient consistency
        pred_grad_x = torch.diff(pred, dim=-1)
        pred_grad_y = torch.diff(pred, dim=-2)
        target_grad_x = torch.diff(target, dim=-1)
        target_grad_y = torch.diff(target, dim=-2)

        gradient_loss = F.mse_loss(pred_grad_x, target_grad_x) + F.mse_loss(pred_grad_y, target_grad_y)

        # Spectral loss (preserve frequency components)
        pred_fft = torch.fft.fft2(pred)
        target_fft = torch.fft.fft2(target)
        spectral_loss = F.mse_loss(torch.abs(pred_fft), torch.abs(target_fft))

        return (self.mse_weight * mse_loss +
                self.gradient_weight * gradient_loss +
                self.spectral_weight * spectral_loss)

In [ ]:
import pandas as pd

class ClimateEmulationModule(pl.LightningModule):
    def __init__(self, model, learning_rate=1e-4):
        super().__init__()
        self.model = model
        self.save_hyperparameters(ignore=['model']) # Save all hyperparameters except the model to self.hparams.<param_name>
        self.criterion = nn.MSELoss()
        self.normalizer = None
        self.val_preds, self.val_targets = [], []
        self.test_preds, self.test_targets = [], []
        self.train_loss_history = []
        self.val_loss_history = []

    def forward(self, x):
        x = x.unsqueeze(1)
        return self.model(x)

    def on_fit_start(self):
        self.normalizer = self.trainer.datamodule.normalizer  # Get the normalizer from the datamodule (see above)

    def training_step(self, batch, batch_idx):
        x, y = batch # Unpack inputs and targets (this is the output of the _getitem_ method in the Dataset above)
        y_hat = self(x)   # Forward pass
        loss = self.criterion(y_hat, y)
        self.train_loss_history.append(loss.item()) # Calculate loss
        self.log("train/loss", loss)  # Log loss for tracking
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.val_loss_history.append(loss.item())
        self.log("val/loss", loss)

        y_hat_np = self.normalizer.inverse_transform_output(y_hat.detach().cpu().numpy())
        y_np = self.normalizer.inverse_transform_output(y.detach().cpu().numpy())
        self.val_preds.append(y_hat_np)
        self.val_targets.append(y_np)

        return loss

    def on_validation_epoch_end(self):
        # Concatenate all predictions and ground truths from each val step/batch into one array
        preds = np.concatenate(self.val_preds, axis=0)
        trues = np.concatenate(self.val_targets, axis=0)
        self._evaluate(preds, trues, phase="val")
        np.save("val_preds.npy", preds)
        np.save("val_trues.npy", trues)
        self.val_preds.clear()
        self.val_targets.clear()

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        y_hat_np = self.normalizer.inverse_transform_output(y_hat.detach().cpu().numpy())
        y_np = y.detach().cpu().numpy()
        self.test_preds.append(y_hat_np)
        self.test_targets.append(y_np)

    def on_test_epoch_end(self):
        # Concatenate all predictions and ground truths from each test step/batch into one array
        preds = np.concatenate(self.test_preds, axis=0)
        trues = np.concatenate(self.test_targets, axis=0)
        self._evaluate(preds, trues, phase="test")
        self._save_submission(preds)
        self.test_preds.clear()
        self.test_targets.clear()

    def configure_optimizers(self):
        return self.model.configure_optimizers()

    def _evaluate(self, preds, trues, phase="val"):
        datamodule = self.trainer.datamodule
        area_weights = datamodule.get_lat_weights()
        lat, lon = datamodule.get_coords()
        time = np.arange(preds.shape[0])
        output_vars = datamodule.output_vars

        for i, var in enumerate(output_vars):
            p = preds[:, i]
            t = trues[:, i]
            p_xr = xr.DataArray(p, dims=["time", "y", "x"], coords={"time": time, "y": lat, "x": lon})
            t_xr = xr.DataArray(t, dims=["time", "y", "x"], coords={"time": time, "y": lat, "x": lon})

            # RMSE
            rmse = np.sqrt(((p_xr - t_xr) ** 2).weighted(area_weights).mean(("time", "y", "x")).item())
            # RMSE of time-mean
            mean_rmse = np.sqrt(((p_xr.mean("time") - t_xr.mean("time")) ** 2).weighted(area_weights).mean(("y", "x")).item())
            # MAE of time-stddev
            std_mae = np.abs(p_xr.std("time") - t_xr.std("time")).weighted(area_weights).mean(("y", "x")).item()

            print(f"[{phase.upper()}] {var}: RMSE={rmse:.4f}, Time-Mean RMSE={mean_rmse:.4f}, Time-Stddev MAE={std_mae:.4f}")
            self.log_dict({
                f"{phase}/{var}/rmse": rmse,
                f"{phase}/{var}/time_mean_rmse": mean_rmse,
                f"{phase}/{var}/time_std_mae": std_mae,
            })

    def _save_submission(self, predictions):
        datamodule = self.trainer.datamodule
        lat, lon = datamodule.get_coords()
        output_vars = datamodule.output_vars
        time = np.arange(predictions.shape[0])

        rows = []
        for t_idx, t in enumerate(time):
            for var_idx, var in enumerate(output_vars):
                for y_idx, y in enumerate(lat):
                    for x_idx, x in enumerate(lon):
                        row_id = f"t{t_idx:03d}_{var}_{y:.2f}_{x:.2f}"
                        pred = predictions[t_idx, var_idx, y_idx, x_idx]
                        rows.append({"ID": row_id, "Prediction": pred})

        df = pd.DataFrame(rows)
        os.makedirs("submissions", exist_ok=True)
        filepath = f"submissions/kaggle_submission_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        df.to_csv(filepath, index=False)
        print(f"✅ Submission saved to: {filepath}")

### ⚡ Training & Evaluation with PyTorch Lightning

This block sets up and runs the training and testing pipeline using **PyTorch Lightning’s `Trainer`**, which abstracts away much of the boilerplate in deep learning workflows.

- **Modular Setup**:
  - `datamodule`: Handles loading, normalization, and batching of climate data.
  - `model`: A convolutional neural network that maps climate forcings to predicted outputs.
  - `lightning_module`: Wraps the model with training/validation/test logic and metric evaluation.

- **Trainer Flexibility**:
  The `Trainer` accepts a wide range of configuration options from `config["trainer"]`, including:
  - Number of epochs
  - Precision (e.g., 16-bit or 32-bit)
  - Device configuration (CPU, GPU, or TPU)
  - Determinism, logging, callbacks, and more

In [ ]:
datamodule = ClimateDataModule(**config["data"])
model = create_climate_unet(
    n_input_channels=len(config["data"]["input_vars"]),
    n_output_channels=len(config["data"]["output_vars"]),
    base_channels=64,       # Start with 64 channels
    depth=4,                # 4 levels of down/upsampling
    bilinear=True,          # Use bilinear upsampling
    use_attention=True,     # Enable spatial attention
    attention_levels=[1, 2], # Apply attention at levels 1 and 2
    dropout_rate=0.1168,
    learning_rate=0.0007325,
    weight_decay=1.75e-5,
    l1_reg=0.0002046
)

# The rest of your training code remains the same
lightning_module = ClimateEmulationModule(model, learning_rate=config["training"]["lr"])
trainer = pl.Trainer(**config["trainer"])
trainer.fit(lightning_module, datamodule=datamodule)

In [ ]:
import matplotlib.pyplot as plt

train_losses = lightning_module.train_loss_history
val_losses = lightning_module.val_loss_history

plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='Training Loss', alpha=0.7)
plt.plot(val_losses, label='Validation Loss', alpha=0.7)
plt.xlabel('Training Steps')
plt.ylabel('MSE Loss')
plt.title('Training and Validation Loss Over Time')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("loss_curve.png", dpi=300)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Assuming you already trained with `trainer.fit(...)`
metrics = trainer.logged_metrics

train_loss = trainer.callback_metrics.get("train/loss")
val_loss = trainer.callback_metrics.get("val/loss")

print(f"Final Train Loss: {train_loss}")
print(f"Final Val Loss: {val_loss}")

# Test model

**IMPORTANT:** Please note that the test metrics will be bad because the test targets have been corrupted on the public Kaggle dataset.
The purpose of testing below is to generate the Kaggle submission file based on your model's predictions, which you can submit to the competition.

In [ ]:
trainer.test(lightning_module, datamodule=datamodule)

### Plotting Utils


In [ ]:
def plot_comparison(true_xr, pred_xr, title, cmap='viridis', diff_cmap='RdBu_r', metric=None, save_path=None):
    fig, axs = plt.subplots(1, 3, figsize=(18, 6))

    vmin = min(true_xr.min().item(), pred_xr.min().item())
    vmax = max(true_xr.max().item(), pred_xr.max().item())

    # Ground truth
    true_xr.plot(ax=axs[0], cmap=cmap, vmin=vmin, vmax=vmax, add_colorbar=True)
    axs[0].set_title(f"{title} (Ground Truth)")

    # Prediction
    pred_xr.plot(ax=axs[1], cmap=cmap, vmin=vmin, vmax=vmax, add_colorbar=True)
    axs[1].set_title(f"{title} (Prediction)")

    # Difference
    diff = pred_xr - true_xr
    abs_max = np.max(np.abs(diff))
    diff.plot(ax=axs[2], cmap=diff_cmap, vmin=-abs_max, vmax=abs_max, add_colorbar=True)
    axs[2].set_title(f"{title} (Difference) {f'- {metric:.4f}' if metric else ''}")

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
    plt.show()


### 🖼️ Visualizing Validation Predictions

This cell loads saved validation predictions and compares them to the ground truth using spatial plots. These visualizations help you qualitatively assess your model's performance.

For each output variable (`tas`, `pr`), we visualize:

- **📈 Time-Mean Map**: The 10-year average spatial pattern for both prediction and ground truth. Helps identify long-term biases or spatial shifts.
- **📊 Time-Stddev Map**: Shows the standard deviation across time for each grid cell — useful for assessing how well the model captures **temporal variability** at each location.
- **🕓 Random Timestep Sample**: Visual comparison of prediction vs ground truth for a single month. Useful for spotting fine-grained anomalies or errors in specific months.

> These plots provide intuition beyond metrics and are useful for debugging spatial or temporal model failures.


In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

# Load validation predictions and ground truth
val_preds = np.load("val_preds.npy")  # shape: [time, channels, y, x]
val_trues = np.load("val_trues.npy")

lat, lon = datamodule.get_coords()
output_vars = datamodule.output_vars
time = np.arange(val_preds.shape[0])

# Compute RMSE per timestep and variable
errors = np.mean((val_preds - val_trues) ** 2, axis=(2, 3))  # shape: [time, variables]
rmse_per_timestep = np.sqrt(errors)

# Number of worst samples to visualize
top_k = 3

for i, var in enumerate(output_vars):
    # Get indices of top-k highest RMSE timesteps
    worst_timestep_indices = np.argsort(rmse_per_timestep[:, i])[::-1][:top_k]

    print(f"\nTop {top_k} highest-error timesteps for '{var}': {worst_timestep_indices}")

    # Wrap each timestep into a plot
    for t_idx in worst_timestep_indices:
        pred_xr = xr.DataArray(val_preds[t_idx, i], dims=["y", "x"], coords={"y": lat, "x": lon})
        true_xr = xr.DataArray(val_trues[t_idx, i], dims=["y", "x"], coords={"y": lat, "x": lon})
        save_path = f"{var}_high_error_t{t_idx}.png"
        plot_comparison(true_xr, pred_xr, f"{var} High-Error Sample Timestep {t_idx}", save_path=save_path)


In [ ]:
def plot_top_k_comparison_grid(true_vals, pred_vals, var_name, lat, lon, top_k_indices, save_path):
    fig, axs = plt.subplots(len(top_k_indices), 3, figsize=(18, 6 * len(top_k_indices)))

    for row, t_idx in enumerate(top_k_indices):
        true_xr = xr.DataArray(true_vals[t_idx], dims=["y", "x"], coords={"y": lat, "x": lon})
        pred_xr = xr.DataArray(pred_vals[t_idx], dims=["y", "x"], coords={"y": lat, "x": lon})
        diff_xr = pred_xr - true_xr

        vmin = min(true_xr.min().item(), pred_xr.min().item())
        vmax = max(true_xr.max().item(), pred_xr.max().item())
        abs_max = np.max(np.abs(diff_xr))

        # Row titles
        true_xr.plot(ax=axs[row, 0], cmap='viridis', vmin=vmin, vmax=vmax, add_colorbar=True)
        axs[row, 0].set_title(f"{var_name} GT (Timestep {t_idx})")

        pred_xr.plot(ax=axs[row, 1], cmap='viridis', vmin=vmin, vmax=vmax, add_colorbar=True)
        axs[row, 1].set_title(f"{var_name} Pred (Timestep {t_idx})")

        diff_xr.plot(ax=axs[row, 2], cmap='RdBu_r', vmin=-abs_max, vmax=abs_max, add_colorbar=True)
        axs[row, 2].set_title(f"{var_name} Diff (Timestep {t_idx})")

    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches="tight")
    plt.close()


In [ ]:
import os
os.makedirs("error_samples", exist_ok=True)

top_k = 3

for i, var in enumerate(output_vars):
    top_k_indices = np.argsort(rmse_per_timestep[:, i])[::-1][:top_k]
    print(f"\nTop {top_k} highest-error timesteps for '{var}': {top_k_indices}")

    save_path = f"error_samples/{var}_top{top_k}_grid.png"
    plot_top_k_comparison_grid(
        val_trues[:, i],
        val_preds[:, i],
        var_name=var,
        lat=lat,
        lon=lon,
        top_k_indices=top_k_indices,
        save_path=save_path
    )


## 🧪 Final Notes

This notebook is meant to serve as a **baseline template** — a starting point to help you get up and running quickly with the climate emulation challenge.

You are **not** required to stick to this exact setup. In fact, we **encourage** you to:

- 🔁 Build on top of the provided `DataModule`.
- 🧠 Use your own model architectures or training pipelines that you’re more comfortable with
- ⚗️ Experiment with ideas  
- 🥇 Compete creatively to climb the Kaggle leaderboard  
- 🙌 Most importantly: **have fun** and **learn as much as you can** along the way

This challenge simulates a real-world scientific problem, and there’s no single "correct" approach — so be curious, experiment boldly, and make it your own!


In [ ]:
!pip install optuna optuna-integration[pytorch_lightning] --quiet

In [ ]:
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import EarlyStopping
import optuna

def objective(trial):
    # --- Suggest hyperparameters ---
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    l1_reg = trial.suggest_float("l1_reg", 1e-6, 1e-3, log=True)
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5)
    hidden_dim = trial.suggest_categorical("hidden_dim", [64, 128, 256])
    cnn_init_dim = trial.suggest_categorical("cnn_init_dim", [32, 64])
    kernel_size = trial.suggest_categorical("kernel_size", [3, 5])

    # --- Build model ---
    model = create_climate_model(
        n_input_channels=len(config["data"]["input_vars"]),
        n_output_channels=len(config["data"]["output_vars"]),
        cnn_init_dim=cnn_init_dim,
        hidden_dim=hidden_dim,
        kernel_size=kernel_size,
        dropout_rate=dropout_rate,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        l1_reg=l1_reg,
    )
    lightning_model = ClimateEmulationModule(model)

    # --- Logger ---
    logger = TensorBoardLogger("optuna_logs", name=f"trial_{trial.number}")

    # --- Early stopping only ---
    early_stopping = EarlyStopping(monitor="val/loss", mode="min", patience=2)

    trainer = pl.Trainer(
        max_epochs=5,
        logger=logger,
        callbacks=[early_stopping],
        enable_progress_bar=False,
        accelerator="auto",
        devices=1,
    )

    # --- Fit ---
    datamodule = ClimateDataModule(**config["data"])
    trainer.fit(lightning_model, datamodule=datamodule)

    # --- Fetch validation loss ---
    val_loss = trainer.callback_metrics.get("val/loss")
    if val_loss is None:
        raise optuna.exceptions.TrialPruned()

    # --- Manual pruning ---
    trial.report(val_loss.item(), step=0)
    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()

    return val_loss.item()


In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10, timeout=3600)  # adjust for time or trials

print("Best trial:")
trial = study.best_trial
print(f"  Value (val/loss): {trial.value}")
print("  Params:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")